<a href="https://colab.research.google.com/github/xinruili07/MiniProject1/blob/master/mini_project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cases_df = pd.read_csv("/content/drive/My Drive/Project1/aggregated_cc_by.csv")
trends_df = pd.read_csv("/content/drive/My Drive/Project1/2020_US_weekly_symptoms_dataset.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [26]:
trends_df.head()

,open_covid_region_code,country_region_code,country_region,sub_region_1,sub_region_1_code,sub_region_2,sub_region_2_code,date,symptom:Abdominal obesity,symptom:Abdominal pain,symptom:Acne,symptom:Actinic keratosis,symptom:Acute bronchitis,symptom:Adrenal crisis,symptom:Ageusia,symptom:Alcoholism,symptom:Allergic conjunctivitis,symptom:Allergy,symptom:Amblyopia,symptom:Amenorrhea,symptom:Amnesia,symptom:Anal fissure,symptom:Anaphylaxis,symptom:Anemia,symptom:Angina pectoris,symptom:Angioedema,symptom:Angular cheilitis,symptom:Anosmia,symptom:Anxiety,symptom:Aphasia,symptom:Aphonia,symptom:Apnea,symptom:Arthralgia,symptom:Arthritis,symptom:Ascites,symptom:Asperger syndrome,symptom:Asphyxia,symptom:Asthma,symptom:Astigmatism,symptom:Ataxia,...,symptom:Throat irritation,symptom:Thrombocytopenia,symptom:Thyroid nodule,symptom:Tic,symptom:Tinnitus,symptom:Tonsillitis,symptom:Toothache,symptom:Tremor,symptom:Trichoptilosis,symptom:Tumor,symptom:Type 2 diabetes,symptom:Unconsciousness,symptom:Underweight,symptom:Upper respiratory tract infection,symptom:Urethritis,symptom:Urinary incontinence,symptom:Urinary tract infection,symptom:Urinary urgency,symptom:Uterine contraction,symptom:Vaginal bleeding,symptom:Vaginal discharge,symptom:Vaginitis,symptom:Varicose veins,symptom:Vasculitis,symptom:Ventricular fibrillation,symptom:Ventricular tachycardia,symptom:Vertigo,symptom:Viral pneumonia,symptom:Visual acuity,symptom:Vomiting,symptom:Wart,symptom:Water retention,symptom:Weakness,symptom:Weight gain,symptom:Wheeze,symptom:Xeroderma,symptom:Xerostomia,symptom:Yawn,symptom:hyperhidrosis,symptom:pancreatitis
0,US-AK,US,United States,Alaska,US-AK,NaN,NaN,2020-01-06,NaN,NaN,NaN,NaN,NaN,12.69,NaN,NaN,NaN,NaN,NaN,12.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.31,NaN,NaN,NaN,...,NaN,NaN,15.86,NaN,NaN,NaN,NaN,NaN,11.90,NaN,NaN,NaN,NaN,18.84,13.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.08,9.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.28,NaN,NaN
1,US-AK,US,United States,Alaska,US-AK,NaN,NaN,2020-01-13,NaN,NaN,NaN,NaN,NaN,9.56,NaN,NaN,NaN,NaN,NaN,12.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.20,NaN,NaN,NaN,...,NaN,NaN,12.24,NaN,NaN,NaN,NaN,NaN,7.84,NaN,NaN,NaN,NaN,16.83,13.01,NaN,NaN,7.65,NaN,NaN,NaN,NaN,NaN,15.30,8.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.26,NaN,NaN
2,US-AK,US,United States,Alaska,US-AK,NaN,NaN,2020-01-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.90,NaN,NaN,NaN,...,NaN,NaN,17.66,NaN,NaN,NaN,NaN,NaN,15.27,NaN,NaN,NaN,NaN,18.40,12.14,NaN,NaN,7.54,NaN,NaN,NaN,NaN,NaN,10.30,16.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.48,NaN,NaN
3,US-AK,US,United States,Alaska,US-AK,NaN,NaN,2020-01-27,NaN,NaN,NaN,NaN,NaN,15.31,7.47,NaN,NaN,NaN,NaN,15.67,NaN,NaN,NaN,NaN,NaN,NaN,7.47,NaN,NaN,NaN,7.65,NaN,NaN,NaN,NaN,NaN,12.03,NaN,NaN,NaN,...,NaN,NaN,12.76,NaN,NaN,NaN,NaN,NaN,9.84,NaN,NaN,NaN,NaN,20.41,17.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.93,NaN,NaN
4,US-AK,US,United States,Alaska,US-AK,NaN,NaN,2020-02-03,NaN,NaN,NaN,NaN,NaN,8.81,NaN,NaN,NaN,NaN,NaN,13.87,NaN,NaN,NaN,NaN,NaN,NaN,8.06,NaN,NaN,NaN,12.56,NaN,NaN,NaN,NaN,NaN,14.24,NaN,NaN,NaN,...,NaN,NaN,16.68,NaN,NaN,NaN,NaN,NaN,10.68,NaN,NaN,NaN,NaN,20.62,16.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.24,10.68,NaN,NaN,10.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.93,NaN,NaN


In [28]:
# Getting all us states
us_states = trends_df.open_covid_region_code.unique()
print(us_states)

['US-AK' 'US-DC' 'US-DE' 'US-HI' 'US-ID' 'US-ME' 'US-MT' 'US-ND' 'US-NE'
 'US-NH' 'US-NM' 'US-RI' 'US-SD' 'US-VT' 'US-WV' 'US-WY']


In [51]:
us_cases = cases_df[cases_df['open_covid_region_code'].isin(us_states)]
us_cases = us_cases.reset_index(drop = True)
us_cases = us_cases[['open_covid_region_code', 'region_name', 'date', 'hospitalized_new']]
print(us_cases[:100])

   open_covid_region_code region_name        date  hospitalized_new
0                   US-WY     Wyoming  2020-03-07               0.0
1                   US-WY     Wyoming  2020-03-08               0.0
2                   US-WY     Wyoming  2020-03-09               0.0
3                   US-WY     Wyoming  2020-03-10               0.0
4                   US-WY     Wyoming  2020-03-11               0.0
..                    ...         ...         ...               ...
95                  US-WY     Wyoming  2020-06-10               1.0
96                  US-WY     Wyoming  2020-06-11               0.0
97                  US-WY     Wyoming  2020-06-12               0.0
98                  US-WY     Wyoming  2020-06-13               1.0
99                  US-WY     Wyoming  2020-06-14               0.0

[100 rows x 4 columns]


In [37]:
def clean_data(df):
  (rows, cols) = df.shape
  print(rows, cols)
  row_threshold = int(rows * 0.05)
  col_threshold = int(cols * 0.05)

  df = df.dropna(axis = 0, thresh = row_threshold)
  df = df.dropna(axis = 1, thresh = col_threshold)
  df = df.reset_index(drop = True)
  (rows, cols) = df.shape
  print(rows, cols)
  return df

In [36]:
us_cases.columns

Index(['open_covid_region_code', 'region_name', 'date', 'hospitalized_new'], dtype='object')

In [44]:
def group_data_by_weeks(df):
  df['date'] = pd.to_datetime(df['date']) - pd.to_timedelta(7, unit='d')
  df = df.groupby(['open_covid_region_code', pd.Grouper(key='date', freq='W-MON')])['hospitalized_new'].sum().reset_index().sort_values(['open_covid_region_code','date'])
  return df

us_cases = group_data_by_weeks(us_cases)

   open_covid_region_code       date  hospitalized_new
0                   US-AK 2020-03-02               0.0
1                   US-AK 2020-03-09               0.0
2                   US-AK 2020-03-16               0.0
3                   US-AK 2020-03-23               7.0
4                   US-AK 2020-03-30              16.0
..                    ...        ...               ...
95                  US-HI 2020-03-16               0.0
96                  US-HI 2020-03-23              12.0
97                  US-HI 2020-03-30               9.0
98                  US-HI 2020-04-06              23.0
99                  US-HI 2020-04-13               8.0

[100 rows x 3 columns]


In [50]:
print("Before cleaning:")
print(trends_df.shape)

trends_df = clean_data(trends_df)

print("After cleaning:")
print(trends_df.shape)

Before cleaning:
(332, 127)
332 127
332 127
After cleaning:
(332, 127)


In [53]:
complete_df = trends_df.merge(us_cases, how='inner', on=['open_covid_region_code', 'date'])
complete_df.head()

,open_covid_region_code,country_region_code,country_region,sub_region_1,sub_region_1_code,date,symptom:Adrenal crisis,symptom:Ageusia,symptom:Allergic conjunctivitis,symptom:Amblyopia,symptom:Amenorrhea,symptom:Angular cheilitis,symptom:Anosmia,symptom:Aphonia,symptom:Ascites,symptom:Asphyxia,symptom:Atheroma,symptom:Auditory hallucination,symptom:Balance disorder,symptom:Beau's lines,symptom:Bleeding on probing,symptom:Burning Chest Pain,symptom:Burning mouth syndrome,symptom:Cataplexy,symptom:Chancre,symptom:Chorea,symptom:Cleft lip and cleft palate,symptom:Clouding of consciousness,symptom:Compulsive hoarding,symptom:Crackles,symptom:Crepitus,symptom:Croup,symptom:Cyanosis,symptom:Depersonalization,symptom:Developmental disability,symptom:Dysautonomia,symptom:Dysgeusia,symptom:Dyspareunia,symptom:Dystonia,symptom:Dysuria,...,symptom:Paranoia,symptom:Pericarditis,symptom:Petechia,symptom:Photodermatitis,symptom:Photophobia,symptom:Photopsia,symptom:Pleurisy,symptom:Polycythemia,symptom:Polydipsia,symptom:Polyuria,symptom:Postural orthostatic tachycardia syndrome,symptom:Ptosis,symptom:Pulmonary edema,symptom:Rectal pain,symptom:Rectal prolapse,symptom:Renal colic,symptom:Rheum,symptom:Round ligament pain,symptom:Rumination,symptom:Sensitivity to sound,symptom:Shallow breathing,symptom:Spasticity,symptom:Splenomegaly,symptom:Sputum,symptom:Stomach rumble,symptom:Stridor,symptom:Subdural hematoma,symptom:Tachypnea,symptom:Tenderness,symptom:Thyroid nodule,symptom:Trichoptilosis,symptom:Upper respiratory tract infection,symptom:Urethritis,symptom:Urinary urgency,symptom:Vasculitis,symptom:Ventricular fibrillation,symptom:Viral pneumonia,symptom:Yawn,region_name,hospitalized_new
0,US-AK,US,United States,Alaska,US-AK,2020-03-09,10.60,NaN,11.61,NaN,12.79,NaN,7.07,7.91,NaN,13.80,13.97,NaN,15.99,13.29,9.42,NaN,11.11,13.29,18.51,8.92,14.13,NaN,NaN,NaN,NaN,34.33,8.58,NaN,NaN,9.76,8.25,9.26,13.13,14.13,...,12.12,NaN,18.34,NaN,9.42,10.60,NaN,11.61,NaN,9.09,NaN,11.11,7.57,7.07,12.28,14.47,13.46,12.28,8.58,10.27,8.08,NaN,11.95,22.72,NaN,13.97,24.06,10.43,12.96,11.11,13.97,46.95,10.27,NaN,9.26,NaN,18.85,10.77,Alaska,0.0
1,US-AK,US,United States,Alaska,US-AK,2020-03-16,11.69,NaN,6.39,NaN,NaN,NaN,15.59,7.95,NaN,11.07,11.85,NaN,9.82,9.97,10.75,9.51,8.42,9.20,18.08,9.51,9.35,NaN,NaN,8.57,NaN,48.00,10.29,7.64,12.16,8.73,8.88,13.09,9.35,12.16,...,10.13,7.64,NaN,NaN,7.64,12.47,NaN,10.60,NaN,7.48,NaN,NaN,9.04,7.33,11.07,12.00,11.22,11.53,11.85,NaN,10.91,7.95,8.88,28.68,9.82,12.94,8.88,9.20,11.22,14.49,9.04,78.71,12.00,NaN,11.53,NaN,24.94,14.34,Alaska,0.0
2,US-AK,US,United States,Alaska,US-AK,2020-03-23,11.15,16.57,NaN,NaN,9.53,NaN,45.17,NaN,NaN,9.39,11.44,6.45,9.97,11.15,9.68,14.08,8.65,11.44,14.67,15.99,10.85,NaN,NaN,8.36,NaN,36.96,8.80,NaN,9.09,7.19,15.99,7.77,11.15,12.47,...,12.47,13.79,14.67,NaN,9.53,10.12,NaN,9.97,NaN,9.09,NaN,7.63,8.21,6.31,10.41,12.32,11.00,12.32,11.44,11.15,9.24,9.53,11.59,29.62,7.33,10.85,9.97,11.88,10.41,7.63,8.36,37.54,8.80,NaN,6.75,7.04,13.79,15.69,Alaska,0.0
3,US-AK,US,United States,Alaska,US-AK,2020-03-30,8.96,8.96,7.11,NaN,8.53,NaN,19.05,NaN,NaN,11.94,7.54,6.54,10.95,12.51,8.11,8.53,10.24,12.37,17.35,11.66,9.95,6.68,NaN,NaN,NaN,25.17,10.95,9.81,7.39,NaN,13.79,7.25,NaN,8.96,...,14.79,10.10,15.07,NaN,8.53,10.24,NaN,8.39,8.11,NaN,NaN,6.83,10.38,NaN,9.81,12.09,NaN,12.65,9.10,7.39,10.81,NaN,7.82,24.32,6.40,12.51,6.97,12.94,9.95,9.38,14.79,24.46,9.81,7.96,7.54,7.11,8.82,17.63,Alaska,1.0
4,US-AK,US,United States,Alaska,US-AK,2020-04-06,10.08,7.35,NaN,NaN,10.80,7.06,6.77,NaN,NaN,9.65,11.52,NaN,10.80,9.65,13.11,7.20,9.22,NaN,13.83,15.85,10.08,NaN,NaN,7.20,NaN,15.99,12.53,7.06,8.07,NaN,11.38,8.07,9.07,12.82,...,9.94,8.93,9.65,NaN,9.51,13.97,NaN,12.68,6.19,5.76,NaN,9.94,9.36,10.37,10.52,10.66,8.21,11.52,9.07,7.49,7.06,7.20,10.08,17.00,10.23,NaN,8.50,9.65,9.22,8.64,13.83,16.28,10.08,NaN,11.09,NaN,9.07,14.12,Alaska,3.0
